In [38]:
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, START, END
from typing import Annotated, TypedDict
from langgraph.types import interrupt, Command
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from langchain_community.tools import tool
import requests
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

In [39]:
load_dotenv()
llm = ChatGroq(model='llama-3.3-70b-versatile', temperature=0.7)

In [40]:
@tool
def get_stock_price(symbol: str) -> dict:
    """
    fetched latest stock price for a given symbol (e.g AAPL, TSLA)
    using alpha vantage api key
    """
    url = (
        "https//www.alphavantage.co/query"
        f"?function=GLOBAL_QUOTE&symbol={symbol}&apikey=7XAD98V632VZEUIB"
    )
    r = requests.get(url)
    return r.json()

@tool
def buy_stock_price(symbol: str, quantity: int) -> dict:
    """
    simulate purchasing a given quantity of a stock symbol

    HUMAN-IN-THE-LOOP
    before confirming the purchase, this tool will interrupt
    and wait for human decision (yes/ anything else)
    """
    decision = interrupt(f"Approve buying {quantity} shares of {symbol}? (yes/no)")

    if isinstance(decision, str) and decision.lower() == 'yes':
        return {
            'status': 'Approved',
            'message': f'Purchased order place for {quantity} shares of {symbol}',
            'symbol': symbol,
            'quantity': quantity
	  }
    else:
        return {
            'status': 'Not Approved',
            'message': f'Order for purchasing shares of {symbol} was declined by human',
            'symbol': symbol,
            'quantity': quantity
	  }
    
tools = [get_stock_price, buy_stock_price]
llm_binded_tools = llm.bind_tools(tools)

In [41]:
class state_class(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [42]:
def chat_bot(state: state_class):
    response = llm_binded_tools.invoke(state['messages'])
    return {'messages': [response]}

tool_node = ToolNode(tools)

In [43]:
checkpointer = MemorySaver()

graph = StateGraph(state_class)

graph.add_node('chatbot', chat_bot)
graph.add_node('tools', chat_bot)

graph.add_edge(START, 'chatbot')
graph.add_conditional_edges('tools', tools_condition)
graph.add_edge('tools', 'chatbot')

chatbot = graph.compile(checkpointer=checkpointer)

In [44]:
# human_message = {
#     'messages': [HumanMessage(content='purchase stock for AAPL price is 990')]
# }

# chatbot.invoke(human_message, config=config)

In [45]:
while True:
    user_input = input('you: ')
    if user_input.lower().strip() in {'exit', 'quit'}:
        print('Goodbye')
        break

    mes = {'messages': [HumanMessage(content=user_input)]}
    config = {'configurable': {'thread_id': 9}}
    result = chatbot.invoke(mes, config=config)

    interrupt = result.get("__interrupt__", [])
    if interrupt:
        prompt_to_human = interrupt[0].value
        print(f"HITL: {prompt_to_human}")
        decision = input('your decision: ').strip().lower()
        result = chatbot.invoke(Command(resume=decision), config=config)

    last_msg = result['messages'][-1]
    print(f"bot: {last_msg.content}\n")


bot: 

bot: 

Goodbye


In [46]:
{'messages': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}, id='f14910be-4c5b-44d6-89e8-57c337648c58'), AIMessage(content="I can help with information about stocks or simulate purchasing a stock. What would you like to know or do? \n\nIf you're interested in getting the latest stock price, I can help with that. Alternatively, if you'd like to simulate buying a stock, I can assist with that as well. \n\nPlease let me know how I can help.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 380, 'total_tokens': 451, 'completion_time': 0.266132758, 'completion_tokens_details': None, 'prompt_time': 0.01863244, 'prompt_tokens_details': None, 'queue_time': 0.014212114, 'total_time': 0.284765198}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_dae98b5ecb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b84d0-dbeb-7803-8f87-63a5e4c79887-0', usage_metadata={'input_tokens': 380, 'output_tokens': 71, 'total_tokens': 451}), HumanMessage(content='hi', additional_kwargs={}, response_metadata={}, id='ebfa5d07-bc40-4872-bf78-76d6668d01e4'), AIMessage(content="I can help with information about stocks or simulate purchasing a stock. What would you like to know or do? \n\nIf you're interested in getting the latest stock price, I can help with that. Alternatively, if you'd like to simulate buying a stock, I can assist with that as well. \n\nPlease let me know how I can help.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 461, 'total_tokens': 532, 'completion_time': 0.099137345, 'completion_tokens_details': None, 'prompt_time': 0.022650201, 'prompt_tokens_details': None, 'queue_time': 0.008328446, 'total_time': 0.121787546}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_68f543a7cc', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b84d1-1973-7b21-a10e-096b1460945a-0', usage_metadata={'input_tokens': 461, 'output_tokens': 71, 'total_tokens': 532}), HumanMessage(content='hi', additional_kwargs={}, response_metadata={}, id='8edefbad-9e7f-4015-899b-c324d5585073'), AIMessage(content="I can help with information about stocks or simulate purchasing a stock. What would you like to know or do? \n\nIf you're interested in getting the latest stock price, I can help with that. Alternatively, if you'd like to simulate buying a stock, I can assist with that as well. \n\nPlease let me know how I can help.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 542, 'total_tokens': 613, 'completion_time': 0.104254647, 'completion_tokens_details': None, 'prompt_time': 0.026580542, 'prompt_tokens_details': None, 'queue_time': 0.008401599, 'total_time': 0.130835189}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_dae98b5ecb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b84d1-48da-72a1-9ab1-a5ec5173ce8a-0', usage_metadata={'input_tokens': 542, 'output_tokens': 71, 'total_tokens': 613}), HumanMessage(content='how', additional_kwargs={}, response_metadata={}, id='786ef109-93c6-4ecc-b5be-667aee918137'), AIMessage(content="I can help with information about stocks or simulate purchasing a stock. What would you like to know or do? \n\nIf you're interested in getting the latest stock price, I can help with that. Alternatively, if you'd like to simulate buying a stock, I can assist with that as well. \n\nPlease let me know how I can help.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 623, 'total_tokens': 694, 'completion_time': 0.100483861, 'completion_tokens_details': None, 'prompt_time': 0.030812428, 'prompt_tokens_details': None, 'queue_time': 0.018225378, 'total_time': 0.131296289}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_dae98b5ecb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b84d1-60a9-78f0-a668-b4cc71aec69f-0', usage_metadata={'input_tokens': 623, 'output_tokens': 71, 'total_tokens': 694}), HumanMessage(content='hi', additional_kwargs={}, response_metadata={}, id='a24a905d-4e18-45b2-9969-9a3a1df0d73d'), AIMessage(content="I can help with information about stocks or simulate purchasing a stock. What would you like to know or do? \n\nIf you're interested in getting the latest stock price, I can help with that. Alternatively, if you'd like to simulate buying a stock, I can assist with that as well. \n\nPlease let me know how I can help.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 704, 'total_tokens': 775, 'completion_time': 0.100238347, 'completion_tokens_details': None, 'prompt_time': 0.034805438, 'prompt_tokens_details': None, 'queue_time': 0.0082966, 'total_time': 0.135043785}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_dae98b5ecb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b84d1-82f7-73f1-840b-191aea9f00a7-0', usage_metadata={'input_tokens': 704, 'output_tokens': 71, 'total_tokens': 775})]}


{'messages': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}, id='f14910be-4c5b-44d6-89e8-57c337648c58'),
  AIMessage(content="I can help with information about stocks or simulate purchasing a stock. What would you like to know or do? \n\nIf you're interested in getting the latest stock price, I can help with that. Alternatively, if you'd like to simulate buying a stock, I can assist with that as well. \n\nPlease let me know how I can help.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 380, 'total_tokens': 451, 'completion_time': 0.266132758, 'completion_tokens_details': None, 'prompt_time': 0.01863244, 'prompt_tokens_details': None, 'queue_time': 0.014212114, 'total_time': 0.284765198}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_dae98b5ecb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b84d0-dbeb-7803-8f87-63a5e4c798